In [1]:
from datetime import datetime, timedelta
import pandas as pd
from misinspect.datasets.payment_transaction import generate_transaction_data
from misinspect.analysis.binary import MisClassifiedTxnAnalyzer
from misinspect.gui.jupyter import MisClassifiedTxnVisualizer

In [2]:
normal = generate_transaction_data(
    1000, 
    (1000, 1010), 
    (1, 3), 
    (datetime.now() - timedelta(days=100), datetime.now() - timedelta(days=5)),
    ["card", "QR", "cash"],
    0,
)

fraud = generate_transaction_data(
    50, 
    (1000, 1010), 
    (1, 3), 
    (datetime.now() - timedelta(days=4), datetime.now()),
    ["card", "QR", "cash"],
    1,
)

dataset = pd.concat([normal, fraud], axis=0).reset_index(drop=True)

In [3]:
analyzer = MisClassifiedTxnAnalyzer(
    dataset=dataset,
    user_id_col = "user_id",
    price_col = "price",
    datetime_col = "use_dt",
    prob_col = "probability",
    label_col = "label",
)

In [4]:
widget = MisClassifiedTxnVisualizer(analyzer)
widget.show()

Dropdown(description='Select threshold: ', options=('0.50', '0.55', '0.60', '0.65', '0.70', '0.75', '0.80', '0…

Select(description='Select misclassification type: ', layout=Layout(width='150px'), options=('FP', 'FN'), valu…

Dropdown(description='Select User Id: ', options=(), value=None)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…